# Introduction

If deferred charge causes bright spots to be stretched out, then we expect that when we measure the moments of the spots, they will become larger.  But how "much" deferred charge do we need for the effect to become noticeable? In particular, since each pixel of the image has a noise to it (question for reader: what are these sources?) can we even detect any changes in the moments of the spots?

It will be difficult to study the relation between deferred charge and the changes to spot size because of a number of factors:
* Real images will have the above mentioned noise.
* The original sizes of the spots, before any deferred charge effects, may be unknown.
* There may be other effects that change the sizes of the spots.
* The exact properties of the deferred charge may be unknown.

For these reasons it may be easier to study this relation using simulated data, where the one has much more control over the images.

## Notebook Description

This notebook is designed to get you started on simulating different deferred charge effects on simulated images of spots. The steps outlined are:

* Set-up a deferred charge simulator.
* Apply the desired deferred charge to an image.

In [3]:
import matplotlib.pyplot as plt

from lsst.eotest.sensor.spotTask import SpotTask, make_ccd_mosaic

from ctisim import BaseOutputAmplifier, FloatingOutputAmplifier
from ctisim import LinearTrap, LogisticTrap
from ctisim import ImageSimulator

# Set-up a Deferred Charge Simulation

You might be wondering what the sources of deferred charge even are.  In fact there are many sources, whose properties are dictated by a number of different physics principles.

The most simple is referred to as "global charge transfer inefficiency". Let's consider that at each pixel transfer, a fixed fraction of the charge is always left behind, perhaps because we are transfer the charge quickly and not given the electrons enough time to move to the next pixel.  This fixed fraction of charge left behind is the charge transfer inefficiency.  For CCDs that require a large number of transfers, this fraction must be very, very small.  If not, since this type of deferred charge occurs at each transfer, the accumulated effect can grow extremely big.  This type of deferred charge will affect all the pixels (hence, the "global" in the name), and pixels that are further from the output amplifier will be more strongly affected.

Consider what happens if, at some location in the CCD, there is a region where electrons have a tendency to get "stuck", taking a little bit of time to escape and join their brethren in being transported to the output amplifier. We refer to these locations as charge traps. Depending on how many electrons get stuck at each location and how long it takes them to get unstuck, the deferred charge caused by this can be quite large or very small.  Only charge that encounters the trap during it's journey to the output amplifir will be affected.

Finally, what happens if instead of electrons being left behind, there is some property of the amplifier that makes it look like the CCD has a lot of deferred charge? These electronic effects can be particularly difficult to distinguish from other deferred charge effects. All pixels will tend to be affected in a similar behavior, though these effects may depend on how much charge is in each pixel, referred to as "signal-dependent" electronic effects.

Anyways, the best way to see how each of the above effects actually changes the CCD images is to play around with them. To do this we will use the tools provided by `ctisim` to setup up a simulator that will take an existing simulated image of spots and smear it out according to our desired deferred charge behavior.

First let's take a look at a simulated spot image.

In [ ]:
## First let's see what the original image looks like
infile = '../examples/data/ITL_simulated_spot_image.fits'

## The simulated image doesn't need as much data reduction
ccd_image = make_ccd_mosaic(infile)

## Get image array
imarr = ccd_image.getArray()

## Plot images side-by-side
fig, ax = plt.subplots(1, 1, figsize=(12, 10), sharey=True)
im0 = ax.imshow(imarr, origin='lower', vmin=10, vmax = 50000.)
ax.set_ylabel('Pixel', fontsize=14)
ax.set_xlabel('Pixel', fontsize=14)
ax.set_title('Original Image', fontsize=16)

## Add a global colorbar
fig.subplots_adjust(right=0.89)
cbar_ax = fig.add_axes([0.9, 0.2, 0.02, 0.6])
cbar = fig.colorbar(im0, cax=cbar_ax)
cbar.set_label('Flux [e-]', fontsize=14)

We'll also run the SpotTask to get a "before" catalog of the spot properties.

In [ ]:
## Set up the task and configure some settings
spottask = SpotTask()
spottask.config.verbose = False
spottask.config.output_dir = '../examples/data/'
spottask.config.characterize_minpixels = 2
spottask.config.characterize_nsig = 4
spottask.config.calibrate_minpixels = 4
spottask.config.characterize_nsig = 25
spottask.config.output_file = 'ITL_simulated_source_catalog.cat'
spottask.config.verbose = True

## Now run the task
spottask.run('ITL_simulated', infile, gains={1.0 for i in range(1, 17)})

Now to play around with some deferred charge settings, starting first with just global CTI.

In [ ]:
## Set a value for the global CTI
b = 1.E-6

## Set up a basic output amplifier (no electronics effects)
output_amps = {amp : BaseOutputAmplifier(1.0) for amp in range(1, 17)}

infile = 'ITL_simulated_spot_image.fits'

## Create an ImageSimulator and process
imsim = ImageSimulator.from_image_fits(infile, output_amps, cti=cti_dict)
imarr_results = imsim.image_readout(infile, outfile='ITL_processed_spot_image.fits')

Above we have created an `ImageSimulator` object that takes in an existing image and processes it according to the global CTI, trap, and output amplifier settings. Behind the scenes the `ImageSimulator` is simulating the read-out process on the image, just like what actually occurs within the CCD.

Now let's see how the spots have changed by running the SpotTask on these new spots that have been subject to the deferred charge we specified.

In [ ]:
## Set up the task and configure some settings
spottask = SpotTask()
spottask.config.verbose = False
spottask.config.output_dir = '../examples/data/'
spottask.config.characterize_minpixels = 2
spottask.config.characterize_nsig = 4
spottask.config.calibrate_minpixels = 4
spottask.config.characterize_nsig = 25
spottask.config.output_file = 'ITL_processed_source_catalog.cat'
spottask.config.verbose = True

## Now run the task
spottask.run('ITL_processed', infile, gains={1.0 for i in range(1, 17)})

Finally, let's create a plot and take a look at the results. Can you notice any change?

In [ ]:
catalog = '../examples/data/ITL_processed_source_catalog.cat'

## Get x/y/z arrays from the source catalog.
X = src['base_SdssShape_x']
Y = src['base_SdssShape_y']
Z = src['base_SdssShape_xx'] # one of the many ways of measuring how bright the spot is

## Scatter plot x/y positions and flux value as the color parameter
fig, ax = plt.subplots(1, 1, figsize=(12.5, 10))
cax = ax.scatter(X, Y, c=Z, marker='o', s=15, vmin=4., vmax=6.)
ax.set_ylabel('Pixel Number', fontsize=12)
ax.set_xlabel('Pixel Number', fontsize=12)
cbar = fig.colorbar(cax)
cbar.set_label('Count', fontsize=12)

## Plot the CCD amplifier geometry (for an ITL sensor)
for i in range(3):
    ax.plot(np.asarray([0, 513*8]), np.asarray([i*2000, i*2000]), 'grey')
for i in range(9):
    ax.plot(np.asarray([513*i, 513*i]), np.asarray([0, 4000]), 'grey')

How high do you need to increase the global CTI to, before changes to the size of the spots are visible? Try playing with the value of the global CTI parameter.  

NOTE: You may have to change the `vmin` and `vmax` parameters in the `ax.scatter()` so that you can see small changes in the spot size.